"""
THIS SCRIPT USES YOLO3 TO LOCALISE THE WORDS AND EXTRACTS CROPS FROM THE
ORIGINAL IMAGE..

It also includes PHOSCNET

"""

In [ ]:
import os
import cv2
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf

#print("\n\t tf:",tf.__version__)
#tf.enable_eager_execution()
# from tensorflow.keras.utils import plot_model
#from yolov3.dataset import Dataset
from yolov3.yolov4 import compute_loss1
from yolov3.utils import load_yolo_weights
from yolov3.configs import *
from evaluate_mAP import get_mAP

print("\n\t YOLO_TYPE:",YOLO_TYPE)

import os
import cv2
import random
import numpy as np
import tensorflow as tf
import pickle
from yolov3.utils import read_class_names, image_preprocess#,draw_bbox
from yolov3.yolov3 import bbox_iou
from yolov3.configs import *
from yolov3.utils import load_yolo_weights, detect_image,  nms, read_class_names #image_preprocesss,
from supportFunctions import draw_bbox, postprocess_boxes


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from yoloModify import yolo33


"""
data exploration
"""
import json
import pickle

"""
    this contains coordinate and words inside BB, No file name mentioned
"""
f = open("./data/tempCord.json")
tempCord1 = json.load(f)
"""
    this contains word and its PHOSC representation
"""
with open('filename.pickle', 'rb') as handle:
    tempText1 = pickle.load(handle)


'''
    more details regarding Dataset1 is in yolo3PhoscModify2.ipynb
'''

from yolov3.dataset import Dataset1
from yolov3.yolov3 import Create_Yolov3
global TRAIN_FROM_CHECKPOINT

gpus = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs {gpus}')
if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

#trainset = Dataset1('train')
testset = Dataset1('test')

if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

steps_per_epoch = len(testset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch


"""
    loading of yolo model
"""

In [ ]:
# from yolov3.yolov4 import Create_YoloPhosc
TRAIN_MODEL_NAME='yolov3_custom'
TRAIN_MODEL_NAME

if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS


if TRAIN_TRANSFER:
    #Darknet = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
    Darknet = Create_Yolov3(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)

    #print("Darknet_weights=",os.path.isfile(Darknet_weights))
    load_yolo_weights(Darknet, Darknet_weights) # use darknet weights
    
#yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

#yolo=Create_Yolov3(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)
yolo=yolo33(input_size=416, channels=3, training=True, CLASSES=YOLO_COCO_CLASSES)

TRAIN_FROM_CHECKPOINT=True
if TRAIN_FROM_CHECKPOINT:
    try:
        print("\n\t load weights!!!")
        yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        print("\n\t loading complete")

    except ValueError:
        print("Shapes are incompatible, transfering Darknet weights")
        TRAIN_FROM_CHECKPOINT = False
'''
if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
    for i, l in enumerate(Darknet.layers):
        layer_weights = l.get_weights()
        if layer_weights != []:
            try:
                yolo.layers[i].set_weights(layer_weights)
            except:
                print("skipping", yolo.layers[i].name)
'''
optimizer = tf.keras.optimizers.Adam()


In [4]:
def validate_step(counter,image_data, target,orgPath,orgImgName,df):
    
        
    pred_result = yolo(image_data, training=False)

    image2=tf.squeeze(image_data)


    # optimizing process
    grid = 3 if not TRAIN_YOLO_TINY else 2
    for i in range(grid):
        
        if i>0:
            continue

        '''
            these r 3 feature maps of size 52*52,26*26,13*13,
                                
        '''
        conv, pred = pred_result[i*2], pred_result[i*2+1]

    
#         print("\n\t\t conv pred:",conv.shape)
#         print("\n\t\t pred pred=",pred.shape)
    

        ''' 
            (xc,yc,w,h,obj,pro) is last dimension of size 6

            pred = (1, 13, 13, 3, 6) is reshaped to 
            pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             
             #2.pred_result: (1, 52, 52, 2325)
            conv: (1, 26, 26, 18)
            pred = (1, 26, 26, 3, 6)
            pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

            image_data= (1, 416, 416, 3)
            pred = (1, 52, 52, 3, 6)
            pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

        '''

    
        
        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
        #print("\n\t 1.pred_bbox pred=",len(pred_bbox))

        pred_bbox = tf.concat(pred_bbox, axis=0)

        #print("\n\t 2.pred_bbox pred=",pred_bbox.shape)

        ''' 
        this removes invalid boxes like boxes having low confidence or 
        which has size greater than image

        '''
        
        label=target[i][0]
        print("\n\t label shape:",label.shape)
        label=label.reshape(-1,775)
        print("\n\t 2.pred_bbox pred=",pred_bbox.shape,"\t label.shape:",label.shape)
        
        if 0:
            accuracyCalc(pred_bbox,label)
        
        
        if 1:

            
            orgImage=cv2.imread(orgPath)
            orgImage2=cv2.imread(orgPath)

            
            #print("\n\t pred_bbox=",len(pred_bbox))
            bboxes = postprocess_boxes(pred_bbox, image2.numpy(),orgImage, YOLO_INPUT_SIZE,0.25)

            
            #print("\n\t pred_bbox=",len(pred_bbox))
            #bboxes = postprocess_boxes(pred_bbox, image2.numpy(), YOLO_INPUT_SIZE,0.25)
            #print("\n\t befor nms bboxes =",bboxes.shape)

            if 0:#i==0:
                label=target[i]
                print("\n\t label:",len(label))
                print("\n\t label0:",label[0].shape)
                print("\n\t label0:",label[1].shape)
                #print("\n\t\t\ label",label[])
                abboxes = postprocess_boxes_Anchors(label[0],pred_bbox, image2.numpy(),416,0.25)


            bboxes = nms(bboxes, 0.1, method='nms')

            #print("\n\t after nms 1.bboxes =",len(bboxes))
            #print("\n\t after nms 2.bboxes =",bboxes)
            #draw_bbox() missing 1 required positional argument: 'bboxes

            
            #draw_bbox(nm,counter,level, image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence=True,Text_colors=(255, 255, 0), rectangle_colors='', tracking=False)
            
            #print("\n\t bboxes len:",len(bboxes))
            
            orgImage=cv2.imread(orgPath)
            
            #resultImage = draw_bbox(nm,counter,i,255*image2.numpy(), bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")
            resultImage = draw_bbox(nm,counter,i,orgImage,orgImgName, bboxes,df, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")

            cv2.imwrite(f'./pipeLine/{orgImgName}.png',resultImage)

            cv2.imwrite(f'./validation/{orgImgName}.png',resultImage)
            #cv2.imwrite(f'./validation/{counter}.png',orgImage)

            #input("check")



In [5]:
#pred_result
forcePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest//"
forceTestImages=os.listdir(forcePath)
forceTestImages[0]

'm02-102.png'

In [6]:
# for counter,(image_data, target,allImagesName) in enumerate(trainset):
from supportFunctions import draw_bbox, postprocess_boxes

imgPaths="/home/aniketag/Documents/phd/yolov5/data//datasets//forms//"

#filePath="./data/data3_phosc.csv"

filePath="/home/aniketag/Documents/phd/yolov5/data/datasets/data_444_april.csv"
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"
presentCrop=os.listdir(testCropPath)

df=pd.read_csv(filePath)
unqImgNames=list(set(df["image_name"]))
len(unqImgNames)

for counter,(image_data, target,allImagesName) in enumerate(testset):
    
    
    nm=str(np.random.randint(10000))+".png"
    nm=str(counter)+".png"
    '''
    print("\n\t image_data=",image_data.shape)    
    print("\n\t target:",target[0][0].shape)
    print("\n\t target:",target[1][0].shape)
    print("\n\t target:",target[2][0].shape)
    '''
    #print("\n\t allImagesName:",allImagesName)
    
    #orgImgName=allImagesName[0].split("forms/")[1]
    
    print("00",allImagesName)
    orgImgName=allImagesName[0].split("phoscTest/")[1]
    #orgImgName=allImagesName[0].split("forms/")[1]
    orgPath=imgPaths+orgImgName
    
    if orgImgName not in unqImgNames:
        continue
    
    else:
        print("orgImgName:",orgImgName)
        #input("check!!!")
    
    if orgImgName in presentCrop:
        print(" folder present hence skipped!!!")
        continue    
    
    validate_step(counter,image_data, target,orgPath,orgImgName,df)
    #input("check!!!")
    #break

00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m01-049.png']
orgImgName: m01-049.png
 folder present hence skipped!!!
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m01-060.png']
orgImgName: m01-060.png
 folder present hence skipped!!!
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m01-079.png']
orgImgName: m01-079.png
 folder present hence skipped!!!
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m01-084.png']
orgImgName: m01-084.png
 folder present hence skipped!!!
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m01-090.png']
orgImgName: m01-090.png
 folder present hence skipped!!!
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m01-095.png']
orgImgName: m01-095.png
 folder present hence skipped!!!
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m01-104.png']
orgImgName: m01-104.png
 folder present hence skipped!!!
00 ['/home/aniketag/Documents/phd/TensorF

00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m03-095.png']
orgImgName: m03-095.png

	 label shape: (1, 52, 52, 3, 775)

	 2.pred_bbox pred= (8112, 171) 	 label.shape: (8112, 775)
 inside post process!!!
 original image name: m03-095.png
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m03-110.png']
orgImgName: m03-110.png

	 label shape: (1, 52, 52, 3, 775)

	 2.pred_bbox pred= (8112, 171) 	 label.shape: (8112, 775)
 inside post process!!!
 original image name: m03-110.png
00 ['/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest/m03-114.png']
orgImgName: m03-114.png

	 label shape: (1, 52, 52, 3, 775)

	 2.pred_bbox pred= (8112, 171) 	 label.shape: (8112, 775)
 inside post process!!!
 original image name: m03-114.png


In [9]:

filePath="./data/data3_phosc.csv"
df1=pd.read_csv(filePath)
df1.head(5)

,Unnamed: 0,image_name,class,width,height,org_x1,org_y1,org_x2,org_y2,text,x,y,w,h,phoc,phos
0,0,l04-087.png,1,2479,3542,361,587,480,652,it,0.17,0.17,0.05,0.02,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0. 0. 0. ...
1,1,l04-087.png,1,2479,3542,507,600,550,648,his,0.21,0.18,0.02,0.01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[1. 0. 1. 0. 1. 0. 0. 2. 0. 0. 0. 1. 0. 0. 0. ...
2,2,l04-087.png,1,2479,3542,603,603,689,652,imagination,0.26,0.18,0.03,0.01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",[ 0. 1. 4. 0. 1. 0. 4. 10. 0. 0. 1. ...
3,3,l04-087.png,1,2479,3542,768,593,1120,686,or,0.38,0.18,0.14,0.03,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. ...
4,4,l04-087.png,1,2479,3542,1164,620,1209,648,was,0.48,0.18,0.02,0.01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0. 0. 2. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. ...


In [1]:
from collections import namedtuple
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

ra = Rectangle(3., 3., 5., 5.)
rb = Rectangle(1., 1., 4., 3.5)
# intersection here is (3, 3, 4, 3.5), or an area of 1*.5=.5

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy

print(area(ra, rb))

0.5


In [ ]:
#filePath="./data/data3_phosc.csv"
#df1=pd.read_csv(filePath)
filterinfDataframe = df1[df1['image_name'] =="g06-018o.png"]
#x1,y1,x2,y2=filterinfDataframe[],filterinfDataframe[],filterinfDataframe[],filterinfDataframe[],

filterinfDataframe1 =filterinfDataframe[["org_x1","org_y1","org_x2","org_y2","text"]]
#g06-018o.png_25_[321, 1098, 404, 1174]
for indx,row in filterinfDataframe1.iterrows():
    x1,y1,x2,y2,text=row
    #print("text:",x1,y1,x2,y2)
    
    ra = Rectangle(321, 1098, 404, 1174)
    rb = Rectangle(x1,y1,x2,y2)

    if area(ra, rb):
        print("text:",x1,y1,x2,y2)
        input("check!!!")
        
        
        

In [ ]:
filterinfDataframe.head(5)

In [2]:
from phoscPipeline import *

2022-04-08 10:40:02.094391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-08 10:40:02.094412: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFl

In [3]:
model=build_model()
MODEL="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/new_IAM_16_"
model.load_weights(MODEL+".h5")
#model=tf.keras.models.load_model(MODEL+".h5")
#print(MODEL)


2022-04-08 10:40:10.829091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-08 10:40:10.829132: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-08 10:40:10.829173: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aniketag-Latitude-7420): /proc/driver/nvidia/version does not exist
2022-04-08 10:40:10.829471: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import os

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data//IAM_test//"

testImgNames=os.listdir(testImgPath)

print(" No of test images:",len(testImgNames))
testImgNames[0]


In [4]:

test_csv_file1="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data//IAM_test_seen.csv"
test_csv_file2="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/IAM_test_unseen.csv"

df_test1=pd.read_csv(test_csv_file1)
df_test2=pd.read_csv(test_csv_file2)

print(df_test1.head(5))
print(df_test2.head(5))

#test_word_label=get_comb_label(list(set(df_test['Word'])))



               Image  Word      Writer
0  n02-154-04-05.png  that  n02-154-04
1  m04-093-02-01.png    it  m04-093-02
2  p03-135-03-07.png   his  p03-135-03
3  n04-044-01-08.png  been  n04-044-01
4  p02-022-07-07.png   the  p02-022-07
               Image       Word      Writer
0  m04-209-01-05.png  surprised  m04-209-01
1  m04-019-06-04.png    lording  m04-019-06
2  n06-128-04-05.png    shallow  n06-128-04
3  n01-020-04-01.png     summit  n01-020-04
4  m03-062-00-02.png  repulsive  m03-062-00


In [5]:
"""
    returns a dictionary having word as a key and PHOSC representation as a value
"""

def get_comb_label(x):
    phos_labels=gen_label(x)
    phoc_labels=gen_phoc_label(x)
    test_labels=dict()
    for x in phos_labels:
        test_labels[x]=np.concatenate((phos_labels[x],phoc_labels[x]),axis=0)
    return test_labels

#train_word_label=get_comb_label(list(set(df_lex['Word'])))

def similarity(x,y):
    return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))



In [6]:
os.getcwd()
from phos_label_generator import *

#test_word_label=get_comb_label(list(set(df_test1['Word'])))
#test_word_label


In [12]:
"""
    test data from PHOSCNET
"""


for indx,row in df_test1.iterrows():
    
    word=row['Word']
    print("indx:",indx," word=",word)
    path2Image=testImgPath+row["Image"]   

    x=img_to_array(load_img(path2Image))

    x = np.expand_dims(x, axis=0)
    y_pred=model.predict(x)
    y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
    #print(y_pred)
    mx=0
    for k in test_word_label:
        temp=similarity(y_pred,test_word_label[k])
        if temp>mx:
            mx=temp
            op=k
    print("")
    print(" original word:",word,"-->",op,mx)

    
"""
path2Image=testImgPath+testImgNames[0]
x=img_to_array(load_img(path2Image))

x = np.expand_dims(x, axis=0)
y_pred=model.predict(x)
y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
print(y_pred)
"""

NameError: name 'df_test1' is not defined

In [ ]:
list(test_word_label.keys())[0]

In [7]:
"""
    gather all words
"""

import re
regex = re.compile('[^a-zA-Z]')
allWords=[]

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/g06-042r.png//"

for indx,imgName in enumerate(os.listdir(testImgPath)):
    
    word=imgName.split("]_")[1]
    word=word.split("_.png")[0]
    word= regex.sub('', word)
    allWords.append(word)
    
allWords=list(set(allWords))
test_word_label=get_comb_label(allWords)
len(allWords)
#allWords

FileNotFoundError: [Errno 2] No such file or directory: '/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/g06-042r.png//'

In [8]:
def predict(path2Image):
    
    x=img_to_array(load_img(path2Image))

    x = np.expand_dims(x, axis=0)
    y_pred=model.predict(x)
    y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
    #print(y_pred)
    mx=0
    for k in test_word_label:
        temp=similarity(y_pred,test_word_label[k])
        if temp>mx:
            mx=temp
            op=k
            
    return op,mx


In [ ]:
import cv2
import os
#m01-060.png

orgImage="m01-060.png" #"g06-042r.png"
testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"+orgImage
savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/final//"
image=cv2.imread(testImgPath+orgImage)
print(" original image shape:",image.shape)

for indx,imgName in enumerate(os.listdir(testCropPath)):
    
    print(" imgName:",imgName)
    word=imgName.split("]_")[1]
    word=word.split("_.png")[0]
    
    cor=imgName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    imgName=imgName.split("_")[0]
    print(" word:",word," coordinate:",x1,y1,x2,y2," imgName:",imgName)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2),(255,0,0), 3)
    
    predWord,score=predict(testImgPath+orgImage)
    
    
cv2.imwrite(savePath+orgImage,image)


 original image shape: (3542, 2479, 3)
 imgName: m01-060.png_44_[1165, 1047, 1443, 1188]_glorious_.png
 word: glorious  coordinate: 1165 1047 1443 1188  imgName: m01-060.png


2022-04-08 10:41:37.949583: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-08 10:41:37.970372: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1804800000 Hz
2022-04-08 10:41:38.215488: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2247838208 exceeds 10% of free system memory.


In [48]:
os.path.isfile(testImagePath+orgImage)
testImagePath

''

In [29]:
import os
testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/g06-042r.png//"
match=0

for indx,imgName in enumerate(os.listdir(testImgPath)):
    
    print(" imgName:",imgName)
    word=imgName.split("]_")[1]
    word=word.split("_.png")[0]
        
    path2Image=testImgPath+imgName   

    x=img_to_array(load_img(path2Image))

    x = np.expand_dims(x, axis=0)
    y_pred=model.predict(x)
    y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
    #print(y_pred)
    mx=0
    for k in test_word_label:
        temp=similarity(y_pred,test_word_label[k])
        if temp>mx:
            mx=temp
            op=k
    print("")
    print(" original word:",word,"-->",op,mx)

    if word==op:
        match+=1
    
print(" Accuracy:",match/(indx+1))

 imgName: g06-042r.png_32_[330, 951, 661, 1005]_douloureux_.png


/tmp/ipykernel_598632/1492369126.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))



 original word: douloureux --> douloureux 945.5403865329882
 imgName: g06-042r.png_16_[1337, 1132, 1518, 1174]_death_.png

 original word: death --> death 994.2022236923053
 imgName: g06-042r.png_9_[328, 1130, 552, 1187]_himself_.png

 original word: himself --> himself 959.5919664068653
 imgName: g06-042r.png_58_[769, 952, 838, 995]_As_.png

 original word: As --> this 888.3643333732588
 imgName: g06-042r.png_26_[1209, 961, 1344, 1003]_1830_.png

 original word: 1830 --> homoeopathy 706.7321201581717
 imgName: g06-042r.png_37_[869, 950, 1039, 1029]_early_.png

 original word: early --> early 953.5104232614891
 imgName: g06-042r.png_38_[1600, 1134, 1704, 1182]_the_.png

 original word: the --> the 980.6751909386699
 imgName: g06-042r.png_29_[1762, 1123, 2089, 1184]_new_.png

 original word: new --> method 811.6437584919781
 imgName: g06-042r.png_56_[1760, 772, 1868, 818]_the_.png

 original word: the --> the 951.7018914102316
 imgName: g06-042r.png_13_[587, 593, 702, 645]_this_.png

 

In [ ]:
"""
http://localhost:8888/notebooks/Untitled.ipynb
http://localhost:8888/notebooks/yolo3PhoscModify3.ipynb#
http://localhost:8888/notebooks/yolo3PhoscModify3ToyData.ipynb
http://localhost:8888/notebooks/yolo3PhoscModify2.ipynb
http://localhost:8888/notebooks/createData_1.ipynb
"""

In [ ]:
import pandas as pd

df=pd.read_csv("./data/data3.csv")
filePath="./data/data3_phosc.csv"
df1=pd.read_csv(filePath)
unqImgNames=list(set(df1["image_name"]))
len(unqImgNames)

In [ ]:
df.head(5)



In [ ]:
#df.head(5)
import os
df=pd.read_csv("./data/data3.csv")

imgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms"

files=os.listdir(imgPath)
files[0]

In [ ]:
"""
modifying for working with single image to join with PHOSCNet

"""

#df.head(5) ./data/data3_phosc.csv
import os
df=pd.read_csv("./data/data3_phosc.csv") # file name BB and word value is available for this
unqImgNames=list(set(df["image_name"]))

imgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//"

files=os.listdir(imgPath)
files[0]


"""
    read any image and predict yolo3 frame work
"""
import numpy as np
import cv2
batch_image = np.zeros((1,416,416, 3), dtype=np.float32)

img=cv2.imread(imgPath+'c03-000b.png')
print("img shape:",img.shape)
img=cv2.resize(img,(416,416))
batch_image[0, :, :, :] = img

def validate_step2(counter,image_data,orgPath,orgImgName):
    
    pred_result = yolo(image_data, training=False)
    image2=tf.squeeze(image_data)

    orgImage=cv2.imread(orgPath+orgImgName)
    #orgImage2=cv2.imread(orgPath)

    # optimizing process
    grid = 3 if not TRAIN_YOLO_TINY else 2
    
    for i in range(grid):
        
        if i>0:
            continue

        '''
            these r 3 feature maps of size 52*52,26*26,13*13,
                                
        '''
        conv, pred = pred_result[i*2], pred_result[i*2+1]
        
        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
        #print("\n\t 1.pred_bbox pred=",len(pred_bbox))

        pred_bbox = tf.concat(pred_bbox, axis=0)

        #print("\n\t 2.pred_bbox pred=",pred_bbox.shape)

        ''' 
        this removes invalid boxes like boxes having low confidence or 
        which has size greater than image

        '''

        #print("\n\t pred_bbox=",len(pred_bbox))
        bboxes = postprocess_boxes(pred_bbox, image2.numpy(),orgImage, YOLO_INPUT_SIZE,0.25)

        bboxes = nms(bboxes, 0.1, method='nms')

        #resultImage = draw_bbox(nm,counter,i,255*image2.numpy(), bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")
        resultImage = draw_bbox(orgImgName,counter,i,orgImage,orgImgName, bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")

        cv2.imwrite(f'./pipeLine/{orgImgName}.png',resultImage)
        #cv2.imwrite(f'./validation/{counter}.png',orgImage)

        #input("check")

batch_image = np.zeros((1,416,416, 3), dtype=np.float32)
orgImgName='c03-000b.png'
orgPath=imgPath

img=cv2.imread(imgPath+orgImgName)
print("1.img shape:",img.shape)

img=cv2.resize(img,(416,416))
batch_image[0, :, :, :] = img

#pred_result = yolo(batch_image, training=False)
#image2=tf.squeeze(batch_image)p

validate_step2(0,batch_image,orgPath,orgImgName)